# Binary Classification with Tweet Embeddings

In [1]:
# import modules
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import KFold
import joblib

## Load data

In [2]:
# Load tweets
tweets = pd.read_excel('COVID19_Dataset-CM-ZB-complete with sources.xlsx')
tweets

,Is_Unreliable,Category,Tweet
0,1,"1, 3, 6, 9",We are living in scary times in Canada. Gov’t ...
1,1,"1, 6, 8, 9","Just as bad in Canada. In fact, our government..."
2,1,"1, 4, 9",It was only a matter of time before the mainst...
3,1,"6, 8",Russia's taking no chances: Foreigners infecte...
4,1,"6, 8, 9",Although there is now a presumptive confirmed ...
...,...,...,...
555,0,NaN,BREAKING: Harvard classes will move online sta...
556,0,NaN,Singularity University is hosting a FREE Virtu...
557,0,NaN,Coronavirus: how does it spread and what are t...
558,0,NaN,Stanford just cancelled classes for the rest o...


In [3]:
# Load tweet embeddings
fname = 'tweet_embed_{}.npy'
fname_NMF = fname.format('NMF')
fname_LDA = fname.format('LDA')
fname_DL = fname.format('DL')
tweet_embeddings_NMF = np.load(fname_NMF)
tweet_embeddings_LDA = np.load(fname_LDA)
tweet_embeddings_DL = np.load(fname_DL)

In [4]:
# list of embeddings to iterate over
embeddings = [tweet_embeddings_NMF, tweet_embeddings_LDA, tweet_embeddings_DL]

# target y
target = np.array(tweets['Is_Unreliable'])

## Binary classification: five-fold CV

In [5]:
# SVC hyperparams to optimize
kernel = ['rbf', 'linear', 'poly', 'sigmoid']
C = [0.001, 0.01, 0.1, 1, 10]

In [6]:
tune_num = int(tweets.shape[0]/5)
tune_num

112

In [7]:
# Compute the folds
num_folds = 5
kf = KFold(n_splits = num_folds, shuffle = True, random_state = 1)
splits = kf.split(tweet_embeddings_NMF) # use any set of embeddings to get train/test indices splits

training_sets = []
testing_sets = []
for train_idx, test_idx in splits:
    training_sets.append(train_idx)
    testing_sets.append(test_idx)

# Construct tuning sets from training sets (20% of data ~ 1 fold) &
# write over training sets (60% of data ~ 3 folds)
tuning_sets = []
for i in range(len(training_sets)):
    train_set = training_sets[i]
    np.random.seed(i)
    tune_idx = np.random.choice(train_set,
                                size = tune_num,
                                replace = False)
    tuning_sets.append(tune_idx)
    new_train_set = train_set[~np.in1d(train_set, tune_idx)]
    training_sets[i] = new_train_set

In [8]:
# Define function to create results dictionary & save models in opt_models dictionary
#
# Inputs: numpy array
# Outputs: dictionary containing model performance stats

def get_results(np_array):
    # Initialize dict to store all model stats
    performance = dict()
    
    # Initialize dict to store optimal models from each fold
    opt_models = dict()

    # Loop over folds
    for i in range(num_folds):
        key1 = 'Fold {}'.format(i+1) # key for the performance dict

        train_idx = training_sets[i]
        test_idx = testing_sets[i]
        tune_idx = tuning_sets[i]

        y_train = target[train_idx]
        y_test = target[test_idx]
        y_tune = target[tune_idx]

        X_train = np_array[train_idx]
        X_test = np_array[test_idx]
        X_tune = np_array[tune_idx]

        # Training & tuning
        models = [] # store list of models in order to retrieve optimal model
        tune_auc = [] # tune based on AUC
        model_dict = dict() # to store model params & performance metric values

        for ker in kernel:
            for el in C:
                # Training
                svc = SVC(C = el, kernel = ker, probability = True)
                svc.fit(X_train, y_train)
                models.append(svc)

                # Tuning
                tune_predict_proba = svc.predict_proba(X_tune)[:,1] # check on this subscripting
                auc = roc_auc_score(y_tune, tune_predict_proba)
                tune_auc.append(auc)

        # Get optimal model based on hyperparameter tuning
        opt_model = models[tune_auc.index(max(tune_auc))] # tune based on AUC
        opt_model_params = opt_model.get_params()
        model_dict['params'] = opt_model_params # store optimal values for model hyperparameters
        opt_models[key1] = opt_model

        # Save training scores
        train_scores = dict() # to store all training scores
        train_predict = opt_model.predict(X_train)
        train_predict_proba = opt_model.predict_proba(X_train)[:,1] # check on this subscripting
        train_scores['auc'] = roc_auc_score(y_train, train_predict_proba)
        train_scores['accuracy'] = accuracy_score(y_train, train_predict)
        train_scores['recall_macro'] = recall_score(y_train, train_predict, average = 'macro')
        train_scores['precision_macro'] = precision_score(y_train, train_predict, average = 'macro')
        train_scores['f1_macro'] = f1_score(y_train, train_predict, average = 'macro')

        # Save training scores dictionary to model dictionary
        model_dict['training'] = train_scores

        # Save tuning scores
        tune_scores = dict() # to store all tuning scores
        tune_predict = opt_model.predict(X_tune)
        tune_predict_proba = opt_model.predict_proba(X_tune)[:,1] # check on this subscripting
        tune_scores['auc'] = roc_auc_score(y_tune, tune_predict_proba)
        tune_scores['accuracy'] = accuracy_score(y_tune, tune_predict)
        tune_scores['recall_macro'] = recall_score(y_tune, tune_predict, average = 'macro')
        tune_scores['precision_macro'] = precision_score(y_tune, tune_predict, average = 'macro')
        tune_scores['f1_macro'] = f1_score(y_tune, tune_predict, average = 'macro')

        # Save tuning scores dictionary to model dictionary
        model_dict['tuning'] = tune_scores

        # Testing
        test_scores = dict() # to store all testing scores
        test_predict = opt_model.predict(X_test)
        test_predict_proba = opt_model.predict_proba(X_test)[:,1]
        test_scores['auc'] = roc_auc_score(y_test, test_predict_proba)
        test_scores['accuracy'] = accuracy_score(y_test, test_predict)
        test_scores['recall_macro'] = recall_score(y_test, test_predict, average = 'macro')
        test_scores['precision_macro'] = precision_score(y_test, test_predict, average = 'macro')
        test_scores['f1_macro'] = f1_score(y_test, test_predict, average = 'macro')

        # Save test scores dictionary to model dictionary
        model_dict['testing'] = test_scores

        # Save model dictionary to overall dictionary
        performance[key1] = model_dict
    
    return performance, opt_models

In [9]:
# get results for NMF embeddings
NMF_results, NMF_models = get_results(tweet_embeddings_NMF)

/Users/caitlinmoroney/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/caitlinmoroney/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/caitlinmoroney/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [10]:
# get results for LDA embeddings
LDA_results, LDA_models = get_results(tweet_embeddings_LDA)

In [11]:
# get results for BERT embeddings
DL_results, DL_models = get_results(tweet_embeddings_DL)

/Users/caitlinmoroney/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/caitlinmoroney/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/caitlinmoroney/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [12]:
DL_results

{'Fold 1': {'params': {'C': 1,
   'break_ties': False,
   'cache_size': 200,
   'class_weight': None,
   'coef0': 0.0,
   'decision_function_shape': 'ovr',
   'degree': 3,
   'gamma': 'scale',
   'kernel': 'rbf',
   'max_iter': -1,
   'probability': True,
   'random_state': None,
   'shrinking': True,
   'tol': 0.001,
   'verbose': False},
  'training': {'auc': 0.9979792966534317,
   'accuracy': 0.9761904761904762,
   'recall_macro': 0.9764605785592739,
   'precision_macro': 0.9761904761904763,
   'f1_macro': 0.9761871013465627},
  'tuning': {'auc': 0.9057692307692307,
   'accuracy': 0.8214285714285714,
   'recall_macro': 0.8128205128205128,
   'precision_macro': 0.8388888888888889,
   'f1_macro': 0.8155467720685112},
  'testing': {'auc': 0.9123086734693877,
   'accuracy': 0.8571428571428571,
   'recall_macro': 0.8571428571428572,
   'precision_macro': 0.858974358974359,
   'f1_macro': 0.8569604086845466}},
 'Fold 2': {'params': {'C': 0.001,
   'break_ties': False,
   'cache_size': 200

In [13]:
# Define function to create results df from nested dictionary
def create_df(input_dict):
    df = pd.DataFrame(input_dict)
    df = df.transpose()
    
    df_params = df['params'].apply(pd.Series)
    
    df_training = df['training'].apply(pd.Series)
    df_training.columns = ['train_' + str(col) for col in df_training.columns]
    
    df_tuning = df['tuning'].apply(pd.Series)
    df_tuning.columns = ['tune_' + str(col) for col in df_tuning.columns]
    
    df_testing = df['testing'].apply(pd.Series)
    df_testing.columns = ['test_' + str(col) for col in df_testing.columns]
    
    final_df = pd.concat([df_training, df_tuning, df_testing, df_params], axis = 1).reset_index()
    final_df = final_df.rename({'index': 'fold_num'}, axis = 1)
    
    return final_df

In [14]:
# Define function to get means for test results from dataframe of full results
def get_test_means(df):
    filter_cols = [col for col in df if col.startswith('test_')]
    df_test = df[filter_cols]
    df_test_mean = pd.DataFrame(df_test.mean(axis = 0)).transpose()
    
    return df_test_mean

In [15]:
dir_prefix = './clf_results/'

# Save NMF embedding results
NMF_full = create_df(NMF_results)
fname = dir_prefix + 'NMF_svm_full_results.csv'
NMF_full.to_csv(fname)

NMF_test_mean = get_test_means(NMF_full)
fname = dir_prefix + 'NMF_svm_testmean_results.csv'
NMF_test_mean.to_csv(fname)

In [16]:
# Save LDA embedding results
LDA_full = create_df(LDA_results)
fname = dir_prefix + 'LDA_svm_full_results.csv'
LDA_full.to_csv(fname)

LDA_test_mean = get_test_means(LDA_full)
fname = dir_prefix + 'LDA_svm_testmean_results.csv'
LDA_test_mean.to_csv(fname)

In [17]:
# Save DL embedding results
DL_full = create_df(DL_results)
fname = dir_prefix + 'DL_svm_full_results.csv'
DL_full.to_csv(fname)

DL_test_mean = get_test_means(DL_full)
fname = dir_prefix + 'DL_svm_testmean_results.csv'
DL_test_mean.to_csv(fname)

In [18]:
# Best folds & test accuracy for NMF, LDA, and DL
emb_names = ['NMF', 'LDA', 'DL']
best_folds = [NMF_full.iloc[NMF_full['test_accuracy'].idxmax()]['fold_num'],
              LDA_full.iloc[LDA_full['test_accuracy'].idxmax()]['fold_num'],
              DL_full.iloc[DL_full['test_accuracy'].idxmax()]['fold_num']]
best_accuracy = [NMF_full.iloc[NMF_full['test_accuracy'].idxmax()]['test_accuracy'],
                 LDA_full.iloc[LDA_full['test_accuracy'].idxmax()]['test_accuracy'],
                 DL_full.iloc[DL_full['test_accuracy'].idxmax()]['test_accuracy']]

best_fold_stats = pd.DataFrame({'embeddings': emb_names,
                                'best_fold_num': best_folds,
                                'test_acc': best_accuracy})
fname = dir_prefix + 'best_folds.csv'
best_fold_stats.to_csv(fname)

In [19]:
# Best fold models for NMF, LDA, and DL

# NMF embeddings
fname = dir_prefix + 'svm_NMF_{}.pkl'.format(NMF_full.iloc[NMF_full['test_accuracy'].idxmax()]['fold_num'])
joblib.dump(NMF_models[NMF_full.iloc[NMF_full['test_accuracy'].idxmax()]['fold_num']],
            fname)

# LDA embeddings
fname = dir_prefix + 'svm_LDA_{}.pkl'.format(LDA_full.iloc[LDA_full['test_accuracy'].idxmax()]['fold_num'])
joblib.dump(LDA_models[LDA_full.iloc[LDA_full['test_accuracy'].idxmax()]['fold_num']],
            fname)

# DL embeddings
fname = dir_prefix + 'svm_DL_{}.pkl'.format(DL_full.iloc[DL_full['test_accuracy'].idxmax()]['fold_num'])
joblib.dump(DL_models[DL_full.iloc[DL_full['test_accuracy'].idxmax()]['fold_num']],
            fname)


['svm_DL_Fold 3.pkl']